In [10]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve

"""
Create Your Own Plasma PIC Simulation (With Python)
Philip Mocz (2020) Princeton Univeristy, @PMocz

Simulate the 1D Two-Stream Instability
Code calculates the motions of electron under the Poisson-Maxwell equation
using the Particle-In-Cell (PIC) method

"""


def getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx ):
	"""
    Calculate the acceleration on each particle due to electric field
	pos      is an Nx1 matrix of particle positions
	Nx       is the number of mesh cells
	boxsize  is the domain [0,boxsize]
	n0       is the electron number density
	Gmtx     is an Nx x Nx matrix for calculating the gradient on the grid
	Lmtx     is an Nx x Nx matrix for calculating the laplacian on the grid
	a        is an Nx1 matrix of accelerations
	"""
	# Calculate Electron Number Density on the Mesh by 
	# placing particles into the 2 nearest bins (j & j+1, with proper weights)
	# and normalizing
	N          = pos.shape[0]
	dx         = boxsize / Nx
	j          = np.floor(pos/dx).astype(int)
	jp1        = j+1
	weight_j   = ( jp1*dx - pos  )/dx
	weight_jp1 = ( pos    - j*dx )/dx
	jp1        = np.mod(jp1, Nx)   # periodic BC
	n  = np.bincount(j[:,0],   weights=weight_j[:,0],   minlength=Nx);
	n += np.bincount(jp1[:,0], weights=weight_jp1[:,0], minlength=Nx);
	n *= n0 * boxsize / N / dx 
	
	# Solve Poisson's Equation: laplacian(phi) = n-n0
	phi_grid = spsolve(Lmtx, n-n0, permc_spec="MMD_AT_PLUS_A")
	
	# Apply Derivative to get the Electric field
	E_grid = - Gmtx @ phi_grid
	
	# Interpolate grid value onto particle locations
	E = weight_j * E_grid[j] + weight_jp1 * E_grid[jp1]
	
	a = -E

	return a
	


def main():
	""" Plasma PIC simulation """
	
	# Simulation parameters
	N         = 40000   # Number of particles
	Nx        = 400     # Number of mesh cells
	t         = 0       # current time of the simulation
	tEnd      = 200      # time at which simulation ends
	dt        = 1       # timestep
	boxsize   = 50      # periodic domain [0,boxsize]
	n0        = 1       # electron number density
	vb        = 3       # beam velocity
	vth       = 1       # beam width
	A         = 0.1     # perturbation
	plotRealTime = True # switch on for plotting as the simulation goes along
	
	# Generate Initial Conditions
	np.random.seed(42)            # set the random number generator seed
	# construct 2 opposite-moving Guassian beams
	pos  = np.random.rand(N,1) * boxsize  
	vel  = vth * np.random.randn(N,1) + vb
	Nh = int(N/2)
	vel[Nh:] *= -1
	# add perturbation
	vel *= (1 + A*np.sin(2*np.pi*pos/boxsize))
	
	# Construct matrix G to computer Gradient  (1st derivative)
	dx = boxsize/Nx
	e = np.ones(Nx)
	diags = np.array([-1,1])
	vals  = np.vstack((-e,e))
	Gmtx = sp.spdiags(vals, diags, Nx, Nx);
	Gmtx = sp.lil_matrix(Gmtx)
	Gmtx[0,Nx-1] = -1
	Gmtx[Nx-1,0] = 1
	Gmtx /= (2*dx)
	Gmtx = sp.csr_matrix(Gmtx)

	# Construct matrix L to computer Laplacian (2nd derivative)
	diags = np.array([-1,0,1])
	vals  = np.vstack((e,-2*e,e))
	Lmtx = sp.spdiags(vals, diags, Nx, Nx);
	Lmtx = sp.lil_matrix(Lmtx)
	Lmtx[0,Nx-1] = 1
	Lmtx[Nx-1,0] = 1
	Lmtx /= dx**2
	Lmtx = sp.csr_matrix(Lmtx)
	
	# calculate initial gravitational accelerations
	acc = getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx )
	
	# number of timesteps
	Nt = int(np.ceil(tEnd/dt))
	
	# prep figure
	fig = plt.figure(figsize=(5,4), dpi=80)
	
	# Simulation Main Loop
	for i in range(Nt):
		# (1/2) kick
		vel += acc * dt/2.0
		
		# drift (and apply periodic boundary conditions)
		pos += vel * dt
		pos = np.mod(pos, boxsize)
		
		# update accelerations
		acc = getAcc( pos, Nx, boxsize, n0, Gmtx, Lmtx )
		
		# (1/2) kick
		vel += acc * dt/2.0
		
		# update time
		t += dt
		%matplotlib
		# plot in real time - color 1/2 particles blue, other half red
		if plotRealTime or (i == Nt-1):
			plt.cla()
			plt.scatter(pos[0:Nh],vel[0:Nh],s=.4,color='blue', alpha=0.5)
			plt.scatter(pos[Nh:], vel[Nh:], s=.4,color='red',  alpha=0.5)
			plt.axis([0,boxsize,-6,6])
			
			plt.pause(0.001)
			
	
	# Save figure
	plt.xlabel('x')
	plt.ylabel('v')
	#plt.savefig('pic.png',dpi=240)
	plt.show()
	    
	return 0
	


if __name__== "__main__":
  main()

Using matplotlib backend: <object object at 0x000002524AF84790>
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg
Using ma

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

#number of nodes in x and y directions
Nx = 10**1
Ny = 10**1
Lx = 10**2
Ly = 10**2
N = 10**2
q = 1
m = 1
e = 1


#note this is only valid when dx = dy
def phi_compute(phi,rho,dx):
    tol = 0.1
    
            
    
    

def getAccel(pos,E,Nx,Ny,Lx,Ly,dt,n0,N):
    dx = Lx/Nx
    dy = Ly/Ny
    j = np.zeros((N,2))
    jp1 = np.zeros((N,2))
    
    j[:,0] = np.floor(pos[:,0]/dx).astype(int)
    j[:,1] = np.floor(pos[:,1]/dy).astype(int)
    jp1 = j + 1
    
    weight_jxjy = (jp1[:,0]*dx - pos[:,0])/dx * (jp1[:,1]*dy - pos[:,1])/dy
    weight_jxp1jy = (pos[:,0] - j[:,0]*dx)/dx * (jp1[:,1]*dy - pos[:,1])/dy
    weight_jxjyp1 = (jp1[:,0]*dx - pos[:,0])/dx * (pos[:,1] - j[:,1]*dy)/dy
    weight_jxp1jyp1 = (pos[:,0] - j[:,0]*dx)/dx * (pos[:,1] - j[:,1]*dy)/dy
    
    #periodic boundary conditions:
    jp1[:,0] = np.mod(jp1[:,0], Nx)
    jp1[:,1] = np.mod(jp1[:,1], Ny)
    
    #smear the particle position across four nearest gridpoints, weighting by distance to each gridpoint 
    xbin = np.linspace(0, Nx, Nx+1)
    ybin = np.linspace(0, Ny, Ny+1)
    
    #count number of particles smeared to each grid point
    n = st.binned_statistic_2d(j[:,0], j[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j,j) grid point
    n += st.binned_statistic_2d(jp1[:,0], jp1[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j+1,j+1) grid point
    n += st.binned_statistic_2d(j[:,0], jp1[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j,j+1) grid point
    n += st.binned_statistic_2d(jp1[:,0], j[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j+1,j) grid point
    
    #weight the contribution  of each particle to each grid point it has been smeared across
    n[j[:,0].astype(int),j[:,1].astype(int)] *= weight_jxjy
    n[jp1[:,0].astype(int),j[:,1].astype(int)] *= weight_jxp1jy
    n[j[:,0].astype(int),jp1[:,1].astype(int)] *= weight_jxjyp1
    n[jp1[:,0].astype(int),jp1[:,1].astype(int)] *= weight_jxp1jyp1
    #n *= n0 * Lx*Ly / N /(dx*dy)
    
    
    
    return n, weight_jxjy


    
    
    
    

SyntaxError: invalid syntax. Perhaps you forgot a comma? (76212938.py, line 20)

In [163]:
N = 10
Nx = 10
Ny = 10
Lx = 10
Ly = 10
E = 1
phi = 1
dt = 1
n0 = 1
pos = np.zeros((N,2))
pos[:5,0] = 1.5
pos[:5,1] = 1.5
pos[5:,0] = 3.5
pos[5:,1] = 3.5
ntest, weighttest = getAccel(pos,E,Nx,Ny,Lx,Ly,dt,n0,N)
print(ntest)
print(weighttest)
xbin = np.linspace(0, Nx, Nx+1)
print(xbin)

[[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.0125 0.0125 0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.0125 0.0125 0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.0125 0.0125 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.0125 0.0125 0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.    ]]
[0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [6]:
import numpy as np
import scipy.stats as st

x = np.random.uniform(size=10)
y = np.random.uniform(size=10)
xbin = [0,0.25,0.5,0.75,1]
ybin = [0,0.25,0.5,0.75,1]

binned = st.binned_statistic_2d(x,y,None,'count',bins=[xbin,ybin],expand_binnumbers=True)
j = np.linspace(0, 12, 13).astype(int)
weight_j = np.ones(13)
weight_j[3] = 6
Nx = 13
n  = np.bincount(j,   weights=weight_j,   minlength=Nx);

print(binned.statistic)

[[1. 0. 1. 1.]
 [0. 0. 0. 0.]
 [1. 0. 2. 1.]
 [1. 0. 1. 1.]]


In [12]:
testroll = np.zeros((10,10))
testroll[:,0] = np.linspace(0,9,10)
np.roll(testroll, 1, axis=0)


array([[9., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [4., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [5., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [6., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [7., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [8., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])